In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
import csv
from datetime import datetime
get_ipython().run_line_magic('matplotlib', 'inline')

from scipy.optimize import minimize

from run_and_query_PLEXOS import run_model, query_model_prices, query_capacity_factor, clear_temp_folder
from update_capacity_bands import update_capacity_bands
from calculate_loss_factor import loss_factor, loss_factor_DUID_CF, callback
from generator_parameters import get_gen_group_params


In [ ]:
# take a dataframe of pricebands and make the capacity in each PB addative
def make_cummulative(curr):
    for i in range(2, 11):
        curr[str(i)] = curr[str(i)] + curr[str(i-1)]
    return curr


In [ ]:
# SET-UP ALL THE NECESSARY PARAMETERS --------------------------------------------------------------------------------------

start_date = '1/7/2018'
end_date = '30/6/2019'


# Create the csv files to write the results to
date_time = datetime.now().strftime("%d.%m.%y_%H.%M.%S")
csv_filename_LF = 'D:/Tools and templates/Static Bidding Optimisation/Output/LossValues_OptimisationRun_' + date_time + '.csv'
with open(csv_filename_LF, 'a', newline='') as csvfile:
    fieldnames = ['Iteration', 'Region', 'LFType', 'Value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
csv_filename_params = 'D:/Tools and templates/Static Bidding Optimisation/Output/Params_OptimisationRun_' + date_time + '.csv'
with open(csv_filename_params, 'a', newline='') as csvfile:
    fieldnames = ['Iteration', 'Station', 'PB2_Param', 'VOLL_Param', 'Spread_Param']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

csv_filenames = [csv_filename_LF, csv_filename_params]


iteration = 0

# Get the list of DUID groupings by station and the total capacity of each DUID
duid_details = pd.read_csv("D:\Tools and templates\Static Bidding Optimisation\Data_Files\DUIDDetails.csv")
duid_list = duid_details.groupby('Station')['DUID'].apply(list).to_dict()
gen_tot_cap = duid_details.set_index('DUID').to_dict()['Max Capacity']
gen_cap_factors = duid_details.set_index('DUID').to_dict()['FY19CF']

# The starting parameters for each station
gen_group_params = get_gen_group_params()

# get the price curve for the start and end dates
price_curve = pd.read_csv("D:\Tools and templates\Static Bidding Optimisation\Data_Files\FY19Prices.csv")
price_curve['Date'] = price_curve['Date'].apply(lambda x : pd.to_datetime(x, format='%d/%m/%Y'))
price_curve = price_curve[price_curve['Date'] >= pd.to_datetime(start_date, format='%d/%m/%Y')]
price_curve = price_curve[price_curve['Date'] <= pd.to_datetime(end_date, format='%d/%m/%Y')]
price_curve = price_curve.drop(columns=['Date'])
for col in price_curve:
    price_curve[col] = price_curve[col].sort_values(ascending=False).values
    price_curve[col] = price_curve[col].apply(lambda x : max(x,1))


# KEEP TRACK OF THE PARAMETERS THROUGHOUT ALL THE OPTIMIZATION RUNS
global curr_gen_group_params
curr_gen_group_params = gen_group_params.copy()


# RESET OFFER QUANTITY
curr = pd.read_csv("D:\Tools and templates\Static Bidding Optimisation\Data_Files\OfferQuantityBase.csv")
make_cummulative(curr)
curr.to_csv("D:\PLEXOS Models\Infigen Beta Model\Bids\Jacks Bidding Files\MarkupPoint.csv", index=False)


# UPDATE CAPACITY BANDS TO MATCH CURRENT PARAMETERS
x0 = []
gen_grouping_to_update = []
for station in gen_group_params:
    for param in gen_group_params[station]:
        x0.append(param)
    gen_grouping_to_update.append(station)
# Update the capacity bands to match the starting parameters
update_capacity_bands(x0, gen_tot_cap, gen_grouping_to_update, duid_list, 
                      curr_gen_group_params, csv_filenames[1], iteration)
iteration+=1


In [ ]:
def record_result(iteration, region, LFType, value):
    with open(csv_filenames[0], 'a', newline='') as csvfile:
        fieldnames = ['Iteration', 'Region', 'LFType', 'Value']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'Iteration':iteration, 'Region':region, 'LFType':LFType, 'Value':value})


In [ ]:
def display_results():
    result_values = []
    results = query_model_prices(start_date, end_date)
    capacity_factor_results = query_capacity_factor(start_date, end_date)

    for region in ['QLD', 'NSW', 'VIC', 'SA']:
        region_list = [region]
        lf = loss_factor(price_curve, region, results, [None, None], True, False)
        lf_cf = loss_factor_DUID_CF(gen_cap_factors, capacity_factor_results, duid_details, region_list, False)
        result_values.append({'Region':region, 'PDCurveLF':lf, 'CFCurveLF':lf_cf})

    df = pd.DataFrame(data=result_values, columns=['Region', 'PDCurveLF', 'CFCurveLF'])
    display(df)
    df.set_index('Region').plot(kind='bar', stacked=True)
    plt.show()
    

In [ ]:

def objective_by_region(params, gens_to_optimize, gen_tot_cap, duid_list, price_curve):
    global iteration
    global curr_gen_group_params
    params = [x-10 for x in params]
    
    # update the gloabl variable which is the current generator group params
    curr_gen_group_params = update_capacity_bands(params, gen_tot_cap, gens_to_optimize['Generators'], 
                                           duid_list, curr_gen_group_params, csv_filenames[1], iteration)

    print('Currently executing model run: ' + str(iteration) + '...', end='\r')
    run_model()
    iteration += 1
    
    results = query_model_prices(start_date, end_date)
   

    # Calculate the loss function for the Price Duration Curve -------------------------------------------------------
    lf = 0
    for region in gens_to_optimize['Regions']:
        region_lf = loss_factor(price_curve, region, results, 
                                gens_to_optimize['PDCurveLimits'], 
                                gens_to_optimize['UseSQRTPrice'], False)
        lf += region_lf
        record_result(iteration, region, 'PD', region_lf)
   

    # Calculate the loss function for the Generator Capacity Factors -------------------------------------------------
    capacity_factor_results = query_capacity_factor(start_date, end_date)
    lf_cf = loss_factor_DUID_CF(gen_cap_factors, capacity_factor_results, duid_details, gens_to_optimize['CFRegions'], False)
    record_result(iteration, gens_to_optimize['CFRegions'][0], 'CF', lf_cf)


    clear_temp_folder()

    return lf+lf_cf


In [ ]:
# Takes a list of regions and a list of fuel types and returns as list of stations as per DUIDDetails.csv
# set input list to None to not filter
# FILTERS BY GENERATORS THAT ARE SET TO 'Bidding' SCENARIO
def get_station_list(regions, fuel_types):
    duid_details = pd.read_csv("D:\Tools and templates\Static Bidding Optimisation\Data_Files\DUIDDetails.csv")    
    gen_list = duid_details[duid_details['BiddingScenario']=='Bidding']
    if regions is not None: gen_list = gen_list[gen_list.Region.isin(regions)]
    if fuel_types is not None: gen_list = gen_list[gen_list.FuelType.isin(fuel_types)]
    return gen_list.Station.unique().tolist()




## optimization_runs
This list defines which optimization runs will occur

It is a **list** of **dictionaries** containing the following keys:


* **Regions**: A list of the regions to include in the PD Curve loss fuction. i.e. ['NSW1', 'VIC1']
* **CFRegions**: The Region to look at to calculate the Capacity Factor for each DUID
* **PDCurveLimits**: A list of two integers, the lower and upper limits of the x-axis of the PD Curve, i.e. [1000, None]
* **Iterations**: The number of iterations of the optimization function to perform. i.e. 200
* **UseSQRTPrice** : either True or False, will determine if the sqrt(price) is used in the Price Duration Curve. Important to use sqrt if there are high (>1000) prices, otherwise curve will be distorted and optimization will fail
* **Generators**: A list of the generators to optimize in this run. i.e. ['CG', 'TALWA1', 'URANQ', 'BW', ]

**Example:**
~~~~
optimization_runs = [{'Regions': ['NSW'], 
                        'PDCurveLimits': [None, None],
                        'CFRegions' : 'NSW',
                        'Iterations': 150,
                        'UseSQRTPrice' : True,
                        'Generators': ['BW', 'ER', 'LD', 'MP', 'VP']}]
~~~~

In [ ]:

# DEFINE WHICH OPTIMIZATIONS TO RUN

optimization_runs = [

    {'Regions': ['QLD', 'NSW'],
     'CFRegions' : ['QLD'],
     'PDCurveLimits': [None, None],
     'Iterations': 200,
     'UseSQRTPrice' : True,
     'Generators': get_station_list(['QLD'], None)
    }
    
# ---------------------------------------------------------------------------------------------------

    
#     {'Regions': ['SA', 'VIC'],
#      'CFRegions' : ['SA'],
#      'PDCurveLimits': [None, None],
#      'Iterations': 160,
#      'UseSQRTPrice' : True,
#      'Generators': get_station_list(['SA'], None)
#     },

#     {'Regions': ['QLD', 'NSW'],
#      'CFRegions' : ['QLD'],
#      'PDCurveLimits': [None, None],
#      'Iterations': 160,
#      'UseSQRTPrice' : True,
#      'Generators': get_station_list(['QLD'], None)
#     },

#     {'Regions': ['VIC', 'NSW', 'SA'],
#      'CFRegions' : ['VIC'],
#      'PDCurveLimits': [None, None],
#      'Iterations': 160,
#      'UseSQRTPrice' : True,
#      'Generators': get_station_list(['VIC'], None)
#     },
    
    
#     {'Regions': ['NSW', 'QLD', 'VIC'],
#      'CFRegions' : ['NSW'],
#      'PDCurveLimits': [None, None],
#      'Iterations': 160,
#      'UseSQRTPrice' : True,
#      'Generators': get_station_list(['NSW'], None)
#     }
    
# ---------------------------------------------------------------------------------------------------
    
#     ,

#     {'Regions': ['VIC', 'SA', 'NSW'],
#      'CFRegions' : ['VIC', 'SA'],
#      'PDCurveLimits': [None, None],
#      'Iterations': 400,
#      'UseSQRTPrice' : True,
#      'Generators': get_station_list(['VIC', 'SA'], None)
#     },

#     {'Regions': ['NSW', 'QLD', 'VIC'],
#      'CFRegions' : ['NSW', 'QLD'],
#      'PDCurveLimits': [None, None],
#      'Iterations': 400,
#      'UseSQRTPrice' : True,
#      'Generators': get_station_list(['NSW', 'QLD'], None)
#     }
    
]

    

In [ ]:

# Before running the optimization check there are no errors with any of the generators listed
# -------------------------------------------------------------------------------------------------------------------------
for optimization_run in optimization_runs:
    try:
        for station in optimization_run['Generators']:
            params = curr_gen_group_params[station]
            duids = duid_list[station]
            for duid in duids:
                capacity = gen_tot_cap[duid]
    except KeyError:
        raise Exception('! Error: Invalid DUID grouping: ' + station)
    for region in optimization_run['Regions']:
        if region not in {'NSW', 'SA', 'VIC', 'TAS', 'QLD'}:
            raise Exception("! Error in Region ID: " + region)
    for region in optimization_run['CFRegions']:
        if region not in {'NSW', 'SA', 'VIC', 'TAS', 'QLD'}:
            raise Exception("! Error in Region ID: " + region)
print("All Region ID and Station IDs valid")
print()
# -------------------------------------------------------------------------------------------------------------------------

# print()
# display_results()    


i=1
# RUN THE OPTIMIZATION ----------------------------------------------------------------------------------------------------
for optimization_run in optimization_runs:
    print('Running optimisation run ' + str(i) + '/' + str(len(optimization_runs)))
    print('Iterations: ' + str(optimization_run['Iterations']))
    print()
    i+=1
    
    x0 = []
    for station in optimization_run['Generators']:
        for param in curr_gen_group_params[station]:
            # 10 is added then subtracted to the params so that the parameters in the optimize.minimize function are equal
            # to the real parameters +10. Done so the minimize function shifts the parameters by enough to have an impact
            x0.append(param+10)
    
    result = minimize(fun=objective_by_region, 
                   x0=x0,
                   args=(optimization_run, gen_tot_cap, duid_list, price_curve),
                   method='nelder-mead', 
                   callback=callback, 
                   options={'maxiter': optimization_run['Iterations'], 'disp': True})

    # record the parameters at the end of a model run:
    pd.Series(curr_gen_group_params).to_csv('D:/Tools and templates/Static Bidding Optimisation/Output/FinalParameters/' 
                                            + 'Final_Parameters_' + datetime.now().strftime("%d.%m.%y_%H.%M.%S") 
                                            + '.csv', header=True)
    print()
    display_results()    

print('Optimisation Finished')

